In [2]:
# Testando API YouTube
import json
import pandas as pd
import seaborn as sns
import numpy as np
import requests
import csv 
from datetime import datetime, timezone

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

In [8]:
# Colocando nossa chave da API do YouTube
import os
from getpass import getpass

api_key = getpass("Insira aqui sua API key da OpenIA:")


In [9]:


def videos_em_alta(n_videos: int, region: str):

    # Definindo os parâmetros que queremos
    params = {
        "part": "snippet",
        "chart": "mostPopular",
        "maxResults": n_videos,  # quantidade de vídeos desejado
        "regionCode": region, # região
        "key": api_key
    }

    # Solicitação à API do YouTube
    url = "https://www.googleapis.com/youtube/v3/videos"
    response = requests.get(url, params=params)
    data = response.json()

    videos = data.get("items",[])

    #caso não há vídeos em alta exibirá a mensagem
    if not videos:
        print("Nenhum vídeo em alta foi encontrado! Ou informações não estão disponíveis")

    # dicionário para salvar as informações
    video_dict = {}

    for video in videos:
        video_id = video["id"]
        video_params = {
            "part": "snippet,statistics",
            "id": video_id,
            "key": api_key
        }

    # Solicitação para obter as informações detalhadas do vídeo
        video_response = requests.get(url, params=video_params)
        video_data = video_response.json()

    # Extraindo as informações 
        video_info = video_data.get("items",[])[0]
        title = video_info["snippet"]["title"]
        views = video_info["statistics"]["viewCount"]
        likes = video_info["statistics"]["likeCount"]
        comments = video_info["statistics"]["commentCount"]
        published_at = video_info["snippet"]["publishedAt"]
        link_video = f"https://www.youtube.com/watch?v={video_id}"


    # Armazenando as infos no dicionário
        video_dict[title] = {
            "views": views,
            "likes": likes,
            "comentarios": comments,
            "publicacao": published_at,
            "link_video": link_video
        }
    # Salvando os dados em um DataFrame
    df = pd.DataFrame.from_dict(video_dict, orient="index")
    df = df.reset_index().rename(columns={'index': 'video'})

    # Convertendo a coluna 'publicacao' para datetime e criando uma nova coluna com o tempo postado
    current_time = datetime.now(timezone.utc)

    df['publicacao'] = pd.to_datetime(df['publicacao'])
    df["horas-publicacao"] = (current_time - df["publicacao"]).dt.total_seconds() / 3600
    df["horas-publicacao"] = df["horas-publicacao"].round()
    return df


In [5]:
# função para fazer update diário dos novos vídeos
def update_df(df, df_up):
    df = pd.concat([df,df_up])
    return df.reset_index(drop=True)


In [10]:
# Abrindo e salvando o df em uma variável
df = videos_em_alta(10, 'BR')
df

,video,views,likes,comentarios,publicacao,link_video,horas-publicacao
0,정국 (Jung Kook) 'Standing Next to You' Official MV,7627883,1806914,121914,2023-11-03 04:00:01+00:00,https://www.youtube.com/watch?v=UNo0TG9LwwI,13.0
1,BOTAFOGO 3 X 4 PALMEIRAS | MELHORES MOMENTOS |...,3192827,93966,8526,2023-11-02 03:12:02+00:00,https://www.youtube.com/watch?v=Z8YtuRj5p20,37.0
2,AO VIVO: GINÁSTICA RÍTMICA | FINAIS VALENDO ME...,342678,12204,8,2023-11-02 21:59:36+00:00,https://www.youtube.com/watch?v=bV-QYBfMkVM,19.0
3,REAÇÃO DO CHRIS ACETO AO AVALIAR RAMON DINO - ...,609453,52785,3733,2023-11-02 13:34:31+00:00,https://www.youtube.com/watch?v=fHSyX7RjDRs,27.0
4,BASTIDORES DA VIRADA ÉPICA | BOTAFOGO 3 X 4 PA...,326358,38978,1083,2023-11-03 00:30:07+00:00,https://www.youtube.com/watch?v=cceoNXztPLw,16.0
5,PASSE DE BATALHA COMPLETO NOVA TEMPORADA! - Fo...,51105,3354,146,2023-11-03 09:30:00+00:00,https://www.youtube.com/watch?v=342vRC-u0AI,7.0
6,The Beatles - Now And Then (Official Audio),5655419,288339,30180,2023-11-02 14:01:47+00:00,https://www.youtube.com/watch?v=AW55J2zE3N4,26.0
7,Hate You,822758,140789,0,2023-11-03 04:00:08+00:00,https://www.youtube.com/watch?v=D1cEMLGvAQk,13.0
8,"SUPERLIVE: CANOAGEM, POLO AQUÁTICO, WRESTLING,...",363707,12295,12,2023-11-02 16:21:16+00:00,https://www.youtube.com/watch?v=w7l5UOJUQuo,24.0
9,Trailer de Jogabilidade do Capítulo 4: Tempora...,128426,11931,1244,2023-11-02 18:39:42+00:00,https://www.youtube.com/watch?v=GoFDpkN0GRM,22.0


In [11]:
df_us = videos_em_alta(10, 'US')


In [12]:

df = update_df(df, df_us)

In [13]:
df

,video,views,likes,comentarios,publicacao,link_video,horas-publicacao
0,정국 (Jung Kook) 'Standing Next to You' Official MV,7627883,1806914,121914,2023-11-03 04:00:01+00:00,https://www.youtube.com/watch?v=UNo0TG9LwwI,13.0
1,BOTAFOGO 3 X 4 PALMEIRAS | MELHORES MOMENTOS |...,3192827,93966,8526,2023-11-02 03:12:02+00:00,https://www.youtube.com/watch?v=Z8YtuRj5p20,37.0
2,AO VIVO: GINÁSTICA RÍTMICA | FINAIS VALENDO ME...,342678,12204,8,2023-11-02 21:59:36+00:00,https://www.youtube.com/watch?v=bV-QYBfMkVM,19.0
3,REAÇÃO DO CHRIS ACETO AO AVALIAR RAMON DINO - ...,609453,52785,3733,2023-11-02 13:34:31+00:00,https://www.youtube.com/watch?v=fHSyX7RjDRs,27.0
4,BASTIDORES DA VIRADA ÉPICA | BOTAFOGO 3 X 4 PA...,326358,38978,1083,2023-11-03 00:30:07+00:00,https://www.youtube.com/watch?v=cceoNXztPLw,16.0
5,PASSE DE BATALHA COMPLETO NOVA TEMPORADA! - Fo...,51105,3354,146,2023-11-03 09:30:00+00:00,https://www.youtube.com/watch?v=342vRC-u0AI,7.0
6,The Beatles - Now And Then (Official Audio),5655419,288339,30180,2023-11-02 14:01:47+00:00,https://www.youtube.com/watch?v=AW55J2zE3N4,26.0
7,Hate You,822758,140789,0,2023-11-03 04:00:08+00:00,https://www.youtube.com/watch?v=D1cEMLGvAQk,13.0
8,"SUPERLIVE: CANOAGEM, POLO AQUÁTICO, WRESTLING,...",363707,12295,12,2023-11-02 16:21:16+00:00,https://www.youtube.com/watch?v=w7l5UOJUQuo,24.0
9,Trailer de Jogabilidade do Capítulo 4: Tempora...,128426,11931,1244,2023-11-02 18:39:42+00:00,https://www.youtube.com/watch?v=GoFDpkN0GRM,22.0


In [14]:
# converendo as colunas 'views', 'likes', 'comentarios' para int
df['views'] = df['views'].astype(int)
df['likes'] = df['likes'].astype(int)
df['comentarios'] = df['comentarios'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   video             20 non-null     object             
 1   views             20 non-null     int32              
 2   likes             20 non-null     int32              
 3   comentarios       20 non-null     int32              
 4   publicacao        20 non-null     datetime64[ns, UTC]
 5   link_video        20 non-null     object             
 6   horas-publicacao  20 non-null     float64            
dtypes: datetime64[ns, UTC](1), float64(1), int32(3), object(2)
memory usage: 1008.0+ bytes


In [15]:
# calculando a quantidade de views por hora
df["views/hour"] = (df["views"]/df["horas-publicacao"]).round(2)
df

,video,views,likes,comentarios,publicacao,link_video,horas-publicacao,views/hour
0,정국 (Jung Kook) 'Standing Next to You' Official MV,7627883,1806914,121914,2023-11-03 04:00:01+00:00,https://www.youtube.com/watch?v=UNo0TG9LwwI,13.0,586760.23
1,BOTAFOGO 3 X 4 PALMEIRAS | MELHORES MOMENTOS |...,3192827,93966,8526,2023-11-02 03:12:02+00:00,https://www.youtube.com/watch?v=Z8YtuRj5p20,37.0,86292.62
2,AO VIVO: GINÁSTICA RÍTMICA | FINAIS VALENDO ME...,342678,12204,8,2023-11-02 21:59:36+00:00,https://www.youtube.com/watch?v=bV-QYBfMkVM,19.0,18035.68
3,REAÇÃO DO CHRIS ACETO AO AVALIAR RAMON DINO - ...,609453,52785,3733,2023-11-02 13:34:31+00:00,https://www.youtube.com/watch?v=fHSyX7RjDRs,27.0,22572.33
4,BASTIDORES DA VIRADA ÉPICA | BOTAFOGO 3 X 4 PA...,326358,38978,1083,2023-11-03 00:30:07+00:00,https://www.youtube.com/watch?v=cceoNXztPLw,16.0,20397.38
5,PASSE DE BATALHA COMPLETO NOVA TEMPORADA! - Fo...,51105,3354,146,2023-11-03 09:30:00+00:00,https://www.youtube.com/watch?v=342vRC-u0AI,7.0,7300.71
6,The Beatles - Now And Then (Official Audio),5655419,288339,30180,2023-11-02 14:01:47+00:00,https://www.youtube.com/watch?v=AW55J2zE3N4,26.0,217516.12
7,Hate You,822758,140789,0,2023-11-03 04:00:08+00:00,https://www.youtube.com/watch?v=D1cEMLGvAQk,13.0,63289.08
8,"SUPERLIVE: CANOAGEM, POLO AQUÁTICO, WRESTLING,...",363707,12295,12,2023-11-02 16:21:16+00:00,https://www.youtube.com/watch?v=w7l5UOJUQuo,24.0,15154.46
9,Trailer de Jogabilidade do Capítulo 4: Tempora...,128426,11931,1244,2023-11-02 18:39:42+00:00,https://www.youtube.com/watch?v=GoFDpkN0GRM,22.0,5837.55


In [34]:
date = df['publicacao'].dt.date[0]

In [36]:
df.to_csv(f'top_10_youtube_{date}.csv', sep=';',index=False)